In [1]:
#@ title pandas numpy
import pandas as pd
import numpy as np

In [2]:
#@title read excel
df = pd.read_excel('/content/beverages_dataset.xlsx')

In [3]:
#@title warning
import warnings
warnings.filterwarnings('ignore')

In [4]:
#@title drive
# from google.colab import drive
# drive.mount('/content/drive')

#cleaning

In [5]:
#@title regular expression
import re

In [6]:
#@title camel case
def camel_case(s):
  s = re.sub(r"[\s.'\-]+", " ", s) # Replace spaces, apostrophes, hyphens with spaces
  s = s.split()
  return "".join([word.capitalize() for word in s])

# Rename the columns to camel case
new_columns = [camel_case(col) for col in df.columns]
df.columns = new_columns

In [7]:
#@title cloumns
df.columns

Index(['ItemCode', 'ItemName', 'ManufactureDate', 'ProductDescription',
       'CustomerSName', 'BuyingDate', 'BuyingTime', 'Quantity', 'Units',
       'UnitPrice(npr)', 'TotalPrice(npr)', 'Category'],
      dtype='object')

# feature

In [8]:
#@title Select specific columns
df_selected = df[['ItemName', 'Quantity','Units', 'UnitPrice(npr)', 'TotalPrice(npr)', 'Category']]

In [9]:
#@title label encoder
from sklearn.preprocessing import LabelEncoder

In [10]:
#@title Encoding categorical values using LabelEncoder
le = LabelEncoder()
for column in df_selected.select_dtypes(include=['object']).columns:
    df_selected[column] = le.fit_transform(df_selected[column])

In [11]:
#@title Keep only numerical values
df_numerical = df_selected.select_dtypes(include=np.number)

In [12]:
#@title Remove outliers (optional, adjust IQR multiplier as needed)
Q1 = df_numerical.quantile(0.25)
Q3 = df_numerical.quantile(0.75)
IQR = Q3 - Q1
df_numerical = df_numerical[~((df_numerical < (Q1 - 1.5 * IQR)) | (df_numerical > (Q3 + 1.5 * IQR))).any(axis=1)]

In [13]:
#@title scikit learn
from sklearn.model_selection import train_test_split

In [14]:
#@title Prepare data for regression models
X = df_numerical.drop('TotalPrice(npr)', axis=1)
y = df_numerical['TotalPrice(npr)']

In [15]:
#@title Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
#@title linear
from sklearn.linear_model import LinearRegression

In [17]:
#@title Train and evaluate Linear Regression model
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
lr_predictions = lr_model.predict(X_test)

In [18]:
#@title ensemble
from sklearn.ensemble import RandomForestRegressor

In [19]:
#@title Train and evaluate Random Forest Regression model
rf_model = RandomForestRegressor()
rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_test)

In [20]:
#@title metrics
from sklearn.metrics import mean_squared_error, r2_score

In [21]:
#@title mse and r2
lr_mse = mean_squared_error(y_test, lr_predictions)
lr_r2 = r2_score(y_test, lr_predictions)

rf_mse = mean_squared_error(y_test, rf_predictions)
rf_r2 = r2_score(y_test, rf_predictions)

In [22]:
#@title Evaluate the models
print("Linear Regression:")
print("Mean Squared Error:", lr_mse)
print("R-squared:", lr_r2)
print("\nRandom Forest Regression:")
print("Mean Squared Error:", rf_mse)
print("R-squared:", rf_r2)

Linear Regression:
Mean Squared Error: 15083882.238144226
R-squared: 0.9889995417456617

Random Forest Regression:
Mean Squared Error: 36445880.45714286
R-squared: 0.9734205438506042


In [23]:
# @title a DataFrame to store the evaluation metrics
metrics_df = pd.DataFrame({
    'Model': ['Linear Regression', 'Random Forest Regression'],
    'MSE': [lr_mse, rf_mse],
    'R-squared': [lr_r2, rf_r2]
})

In [24]:
#@title plotly
import plotly.express as px

In [25]:
# @title a bar chart using Plotly to compare MSE
fig_mse = px.bar(metrics_df, x='Model', y='MSE', title='Comparison of Mean Squared Error (MSE)')
fig_mse.show()

In [26]:
# @title a bar chart using Plotly to compare R-squared
fig_r2 = px.bar(metrics_df, x='Model', y='R-squared', title='Comparison of R-squared')
fig_r2.show()

# Advanced Data Analytics Visualizations and DataFrames with Plotly

In [27]:
import plotly.figure_factory as ff
import plotly.graph_objects as go

In [28]:
# @title Total Price by Category with Average Unit Price
df_category_agg = df.groupby('Category').agg({'TotalPrice(npr)': 'sum', 'UnitPrice(npr)': 'mean'})
fig_category_agg = px.bar(df_category_agg, x=df_category_agg.index, y='TotalPrice(npr)',
                          title='Total Price and Average Unit Price by Category',
                          labels={'TotalPrice(npr)': 'Total Price (NPR)', 'UnitPrice(npr)': 'Average Unit Price (NPR)'},
                          hover_data=['UnitPrice(npr)'])
fig_category_agg.show()

In [29]:
# @title Distribution of Quantity by Category with Box Plot
fig_quantity_box = px.box(df, x='Category', y='Quantity', title='Distribution of Quantity by Category')
fig_quantity_box.show()

In [30]:
#@title Scatter Plot of Total Price vs. Quantity with Category as Color
fig_scatter_category = px.scatter(df, x='Quantity', y='TotalPrice(npr)', color='Category',
                                 title='Total Price vs. Quantity by Category',
                                 labels={'Quantity': 'Quantity', 'TotalPrice(npr)': 'Total Price (NPR)'})
fig_scatter_category.show()

In [31]:
# @title Heatmap of Correlation Matrix with Annotated Values
fig_heatmap_corr = px.imshow(df_numerical.corr(), text_auto=True, title='Correlation Matrix of Numerical Features')
fig_heatmap_corr.show()

In [32]:
# @title Pie Chart of Category Proportions
df_category_counts = df.groupby('Category')['ItemName'].count()
fig_pie_category = px.pie(df_category_counts, values='ItemName', names=df_category_counts.index,
                          title='Category Proportions')
fig_pie_category.show()


In [33]:
#@title Histogram of Unit Prices
fig_hist_unitprice = px.histogram(df, x='UnitPrice(npr)', nbins=20, title='Distribution of Unit Prices')
fig_hist_unitprice.show()

In [34]:
# @title Treemap of Total Price by Item Name and Category
fig_treemap = px.treemap(df, path=['Category', 'ItemName'], values='TotalPrice(npr)',
                         title='Total Price by Item Name and Category')
fig_treemap.show()

In [35]:
#@title Parallel Coordinates Plot for Exploring Relationships Between Features
fig_parallel_coords = px.parallel_coordinates(df_numerical, color='TotalPrice(npr)',
                                             title='Parallel Coordinates Plot of Numerical Features')
fig_parallel_coords.show()

In [36]:
# @title Sunburst Chart for Hierarchical Relationships of Category and Item Name
fig_sunburst = px.sunburst(df, path=['Category', 'ItemName'], values='TotalPrice(npr)',
                           title='Hierarchical Breakdown of Total Price by Category and Item Name')
fig_sunburst.show()

In [37]:
# @title Group by Category and calculate total sales and average unit price
category_sales = df.groupby('Category').agg({'TotalPrice(npr)': 'sum', 'UnitPrice(npr)': 'mean'})

# Rename columns for better readability
category_sales = category_sales.rename(columns={'TotalPrice(npr)': 'TotalSales', 'UnitPrice(npr)': 'AverageUnitPrice'})

# Sort by TotalSales in descending order
category_sales = category_sales.sort_values('TotalSales', ascending=False)

print("Category Sales and Average Unit Price:")
display(category_sales)

# bar chart
fig = px.bar(
    category_sales,
    x=category_sales.index,
    y='TotalSales',
    title='Total Sales by Category',
    labels={'x': 'Category', 'TotalSales': 'Total Sales (NPR)'},
    text_auto=True
)
fig.show()



Category Sales and Average Unit Price:


,TotalSales,AverageUnitPrice
Category,,
Alcohol,34522400,274.891192
Whisky,1042800,330.000000
Vodka,495000,330.000000
Wine,316800,231.000000


In [38]:
#@title Group by ItemName and calculate total quantity sold
item_sales = df.groupby('ItemName').agg({'Quantity': 'sum'})

# Sort by total quantity sold in descending order
item_sales = item_sales.sort_values('Quantity', ascending=False)

print("Item Sales:")
display(item_sales.head(10))

# a horizontal bar chart
fig = px.bar(item_sales.head(10),
             x='Quantity',
             y=item_sales.index[:10],
             orientation='h',
             labels={'Quantity': 'Total Quantity Sold', 'ItemName': 'Item Name'},
             title='Top 10 Items by Quantity Sold')

fig.show()


Item Sales:


,Quantity
ItemName,
Kathmandu IPA,4100
Sherpa Lager,4000
Kathmandu Pilsner,3900
Gorkha Wheat Beer,3400
Gorkha IPA,3400
J.P. Chenet Red Wine,3000
Gorkha Dark Lager,2700
Himalayan Lager,2500
Sherpa IPA,2500


In [39]:
# @title Group by Category and ItemName and calculate total sales
category_item_sales = df.groupby(['Category', 'ItemName']).agg({'TotalPrice(npr)': 'sum'})

# Sort by total sales in descending order
category_item_sales = category_item_sales.sort_values('TotalPrice(npr)', ascending=False)

print("Category and Item Sales:")
display(category_item_sales.head(10))

# using a treemap
fig = px.treemap(category_item_sales.reset_index(), path=['Category', 'ItemName'], values='TotalPrice(npr)',
                  title='Sales Distribution by Category and Item')
fig.show()

Category and Item Sales:


TotalPrice(npr)
Category ItemName                             
Alcohol  Kathmandu IPA                 1258000
         Gorkha IPA                    1034000
         Kathmandu Pilsner              902000
         Sherpa Lager                   872000
         Gorkha Wheat Beer              817000
         Sherpa IPA                     764000
         Kathmandu Strong Ale           675000
         Gorkha Dark Lager              669000
         Annapurna Golden Ale           660000
         Kathmandu Brown Ale            656000

In [40]:
# @title Group by Category and calculate the average unit price for each category
category_avg_price = df.groupby('Category').agg({'UnitPrice(npr)': 'mean'})

print("Average Unit Price by Category:")
display(category_avg_price)

# y using a bar chart
fig = px.bar(category_avg_price,
             x=category_avg_price.index,
             y='UnitPrice(npr)',
             labels={'UnitPrice(npr)': 'Average Unit Price (NPR)', 'Category': 'Category'},
             title='Average Unit Price by Category')


fig.show()

Average Unit Price by Category:


,UnitPrice(npr)
Category,
Alcohol,274.891192
Vodka,330.000000
Whisky,330.000000
Wine,231.000000


In [41]:
# @title Group by ItemName and calculate the total revenue generated by each item
item_revenue = df.groupby('ItemName').agg({'TotalPrice(npr)': 'sum'})

print("Item Revenue:")
display(item_revenue.head(10))

#  a pie chart
fig = px.pie(item_revenue.head(10),
             names=item_revenue.index[:10],
             values='TotalPrice(npr)',
             title='Top 10 Item Revenue Distribution',
             labels={'TotalPrice(npr)': 'Revenue (NPR)', 'ItemName': 'Item Name'})

fig.show()


Item Revenue:


,TotalPrice(npr)
ItemName,
Absolut Vodka,99000
Annapurna Ale,184000
Annapurna Amber Ale,135000
Annapurna Brown Ale,486000
Annapurna Brown Lager,135000
Annapurna Dark Ale,200000
Annapurna Dark Lager,250000
Annapurna Golden Ale,660000
Annapurna IPA,360000


In [42]:
# @title Calculate the total sales and average unit price for each item in each category
category_item_analysis = df.groupby(['Category', 'ItemName']).agg({'TotalPrice(npr)': 'sum', 'UnitPrice(npr)': 'mean'})

# Display the DataFrame
print("Category and Item Analysis:")
display(category_item_analysis.head(10))

# Visualize category and item analysis using a treemap
fig = px.treemap(category_item_analysis.reset_index(),
                 path=['Category', 'ItemName'],
                 values='TotalPrice(npr)',
                 color='UnitPrice(npr)',
                 hover_data=['UnitPrice(npr)'],
                 title='Sales and Average Unit Price by Category and Item')

fig.show()


Category and Item Analysis:


TotalPrice(npr)  UnitPrice(npr)
Category ItemName                                              
Alcohol  Annapurna Ale                   184000      230.000000
         Annapurna Amber Ale             135000      270.000000
         Annapurna Brown Ale             486000      270.000000
         Annapurna Brown Lager           135000      270.000000
         Annapurna Dark Ale              200000      250.000000
         Annapurna Dark Lager            250000      250.000000
         Annapurna Golden Ale            660000      265.000000
         Annapurna IPA                   360000      300.000000
         Annapurna Lager                 438000      233.333333
         Annapurna Pilsner               437000      243.333333

In [43]:
# @title the correlation matrix between different variables (e.g., quantity, unit price, total price) ?!!!??!?!?!?!
# correlation_matrix = df[['Quantity', 'UnitPrice(npr)', 'TotalPrice(npr)']].corr()

# print("Correlation Matrix:")
# display(correlation_matrix)

# plt.figure(figsize=(8, 6))
# sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
# plt.title('Correlation Matrix')
# plt.show()

## this is matplot

In [44]:
#@title Calculate the correlation matrix between different variables (e.g., quantity, unit price, total price)
correlation_matrix = df[['Quantity', 'UnitPrice(npr)', 'TotalPrice(npr)']].corr()

print("Correlation Matrix:")
display(correlation_matrix)

# Visualize the correlation matrix using a heatmap with Plotly
fig = ff.create_annotated_heatmap(
    z=correlation_matrix.values,
    x=list(correlation_matrix.columns),
    y=list(correlation_matrix.index),
    annotation_text=correlation_matrix.round(2).values,
    colorscale='Viridis'
)

fig.show()

Correlation Matrix:


,Quantity,UnitPrice(npr),TotalPrice(npr)
Quantity,1.000000,-0.376696,0.505291
UnitPrice(npr),-0.376696,1.000000,0.407425
TotalPrice(npr),0.505291,0.407425,1.000000


In [45]:
#@title Group by Category and calculate the total number of items sold in each category
category_item_count = df.groupby('Category').agg({'ItemName': 'count'})

print("Number of Items Sold by Category:")
display(category_item_count)

# the number of items sold by category using a histogram
fig = px.histogram(df,
                   x='Category',
                   title='Distribution of Items Sold by Category',
                   labels={'Category': 'Category', 'count': 'Number of Items Sold'},
                   category_orders={'Category': category_item_count.index.tolist()})
fig.show()

Number of Items Sold by Category:


,ItemName
Category,
Alcohol,193
Vodka,3
Whisky,5
Wine,2
